# DGEB Final Project Notebook: Traditional Models for Convergent Enzyme Classification

## Authors: Mahbuba Tasmin & Bryn Reimer  
Course: CS 690U — Spring 2025  
Date: April 2025

## Objective

We aim to benchmark the performance of the following traditional approaches:
- **Logistic regression** trained on one-hot encoded DNA sequences (512 bp max).
- **Logistic regression** trained on $k$-mer count features (e.g., 4-mers).
- **BLAST**-based nearest-neighbor prediction using top alignment match from training data.
- **Cross-validation** to assess generalization capability on the training set.

The remainder of this notebook is structured as follows:
1. Dataset loading and preprocessing
2. One-hot encoding pipeline + logistic regression
3. $k$-mer encoding variant
4. Cross-validation evaluation
5. BLAST-based baseline
6. Comparative performance summary and interpretations




## download dataset

In [12]:
import pandas as pd
from datasets import load_dataset
from datasets import DatasetDict

import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split


In [5]:


dataset = load_dataset(
    "parquet",
    data_files={
        "train": [
            "ec_classification_dna_data/test-00000-of-00001.parquet",
            "ec_classification_dna_data/train-00000-of-00001.parquet",
        ],
        "test": "./ec_classification_dna_data/test-00000-of-00001.parquet",
    }
)


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
#explore dataset

# Peek at the structure
print(dataset)
print(dataset["train"][0])


DatasetDict({
    train: Dataset({
        features: ['Entry', 'Label', 'Sequence'],
        num_rows: 640
    })
    test: Dataset({
        features: ['Entry', 'Label', 'Sequence'],
        num_rows: 128
    })
})
{'Entry': 'A0A0H2XEA6', 'Label': '1.14.14.18', 'Sequence': 'TCAAGGACGTGCCTCCGCTGCCAGGCAGGTGTGGAAATGGGCAAACATGGCCTGCGCGCCGGCGATGGCGTCGGCACGCGCCGCTGCGCTCTGCAGGCGTTGCTCGAGCACCGCCTGAAAGCGCCGCCAGCCGGCCGGGTCTTCGTCGGCCAGCTCGAAATAGTGCAATGCATGCGCCAGGCCCGGCTGACGTTTGCGCAGCATGCGTGCAATCACCCGGCCACCCAGCTGAGAGCCTTCGATGACATACAGCATGCCCCAGCGCGCCGCCTCGCTGCTGGCCGGCGGCGGCACGGCGGCATCGACGGGTTGGCCGAGCACGCGCAGATCCTCCCGCAAGGCCGGCACCCGGCGGCGGTACTGCCAGCCGCTGCCCACCAGCGTCACCAGCCAGTCGCTCAGCTGTTCTTCGAAACCGGCCAGCAGCCGGTGATGCCGGCGCAGCACCTGCGCGTAGGTATCGGCGTCGATGTGGCCCTGCCCCAGCGCCTGCATCAG'}


## one-hot encoding based

In [14]:
##one hot encoding - 512 bp (we can expand)
def one_hot_encode_dna(seq, max_len=512):
    bases = "ACGT"
    base_to_idx = {b: i for i, b in enumerate(bases)}
    encoding = np.zeros((max_len, len(bases)))
    for i, base in enumerate(seq[:max_len]):
        if base in base_to_idx:
            encoding[i, base_to_idx[base]] = 1.0
    return encoding.flatten()  # shape: 512 * 4 = 2048




In [13]:
## load dataset and convert to pandas
# Convert Hugging Face DatasetDict to pandas
df_train = pd.DataFrame(dataset["train"])
df_test = pd.DataFrame(dataset["test"])

print("Train size:", len(df_train), "Test size:", len(df_test))
print(df_train.head())


Train size: 640 Test size: 128
        Entry       Label                                           Sequence
0  A0A0H2XEA6  1.14.14.18  TCAAGGACGTGCCTCCGCTGCCAGGCAGGTGTGGAAATGGGCAAAC...
1      Q5AD07    1.15.1.1  ATGAAGTATTTGTCCATTTTCTTACTTGCTACTTTTGCTTTGGCTG...
2      O74831    1.16.3.1  ATGCAGTCTTTGCGAGCAGCCTTTCGCAGACGAACCCCAATTTTTT...
3      O46310    1.17.4.1  ATGAAACCGGTTGCGGCTGGCGCCGAGGTGCTGCCGGCGGACAAGG...
4      Q8PU58    1.5.98.3  ATGTCTGGAATAATTGATAGCTATATACCGGTTGCCATATTTCTTG...


In [15]:
## apply encoding to all seq
# Apply to training and test data
X_train = np.stack(df_train["Sequence"].map(one_hot_encode_dna))
X_test = np.stack(df_test["Sequence"].map(one_hot_encode_dna))

# Check shape
print("X_train shape:", X_train.shape)  # Expect (640, 2048)


X_train shape: (640, 2048)


In [16]:
## encode ec numbers as integer labels

le = LabelEncoder()
y_train = le.fit_transform(df_train["Label"])
y_test = le.transform(df_test["Label"])

# Optional: Show label classes
print("Classes:", le.classes_)


Classes: ['1.14.14.18' '1.15.1.1' '1.16.3.1' '1.17.4.1' '1.5.98.3' '1.8.3.2'
 '2.1.1.354' '2.1.1.360' '2.1.1.37' '2.1.1.72' '2.1.1.77' '2.1.1.86'
 '2.1.3.15' '2.3.1.225' '2.3.1.269' '2.3.1.48' '2.3.1.51' '2.3.2.23'
 '2.3.2.26' '2.3.2.27' '2.3.2.31' '2.4.1.109' '2.4.1.198' '2.4.2.31'
 '2.5.1.18' '2.7.1.107' '2.7.1.21' '2.7.1.33' '2.7.1.67' '2.7.10.1'
 '2.7.10.2' '2.7.11.1' '2.7.11.25' '2.7.13.3' '2.7.4.3' '2.7.6.1'
 '2.7.7.108' '2.7.7.19' '2.7.7.48' '2.7.7.49' '2.7.7.6' '2.7.7.65'
 '2.7.7.7' '2.7.8.7' '2.8.1.13' '3.1.1.1' '3.1.1.3' '3.1.1.32' '3.1.1.4'
 '3.1.1.96' '3.1.11.2' '3.1.11.6' '3.1.12.1' '3.1.13.4' '3.1.21.10'
 '3.1.21.4' '3.1.26.3' '3.1.26.4' '3.1.26.5' '3.1.3.16' '3.1.3.18'
 '3.1.3.2' '3.1.3.3' '3.1.3.48' '3.1.3.5' '3.1.4.12' '3.1.4.35' '3.1.4.4'
 '3.1.4.52' '3.2.1.1' '3.2.1.14' '3.2.1.17' '3.2.1.18' '3.2.1.22'
 '3.2.1.23' '3.2.1.35' '3.2.1.39' '3.2.1.4' '3.2.1.52' '3.2.1.55'
 '3.2.1.78' '3.2.1.8' '3.2.1.96' '3.2.2.22' '3.2.2.6' '3.4.19.12'
 '3.4.21.105' '3.4.22.49' '3.4.23.3

In [ ]:
## train log reg
clf = LogisticRegression(
    multi_class='multinomial',
    solver='saga',  # better for multinomial + large data
    max_iter=1000,
    n_jobs=-1,
    verbose=1
)
clf.fit(X_train, y_train)


.00774938
Epoch 278, change: 0.00802632
Epoch 279, change: 0.02109597
Epoch 280, change: 0.01944719
Epoch 281, change: 0.01083489
Epoch 282, change: 0.00198825
Epoch 283, change: 0.00265499
Epoch 284, change: 0.00523688
Epoch 285, change: 0.02410493
Epoch 286, change: 0.01051703
Epoch 287, change: 0.01796740
Epoch 288, change: 0.01975695
Epoch 289, change: 0.00055037
Epoch 290, change: 0.01675781
Epoch 291, change: 0.01345648
Epoch 292, change: 0.01774485
Epoch 293, change: 0.00371184
Epoch 294, change: 0.00944973
Epoch 295, change: 0.00994424
Epoch 296, change: 0.00454848
Epoch 297, change: 0.01219804
Epoch 298, change: 0.01461690
Epoch 299, change: 0.00327917
Epoch 300, change: 0.01568474
Epoch 301, change: 0.01067470
Epoch 302, change: 0.00463764
Epoch 303, change: 0.00370793
Epoch 304, change: 0.00146539
Epoch 305, change: 0.01473640
Epoch 306, change: 0.01535496
Epoch 307, change: 0.01497194
Epoch 308, change: 0.01596771
Epoch 309, change: 0.01857539
Epoch 310, change: 0.02243267


In [ ]:
## evaluate model
y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Macro F1-score:", f1_score(y_test, y_pred, average='macro'))

# Optional: Full classification report
print(classification_report(y_test, y_pred, target_names=le.classes_))


In [ ]:
## save encoding data for reuse 

np.savez("convenz_onehot_data.npz", X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, classes=le.classes_)


In [ ]:
# visualize model coefficients
import matplotlib.pyplot as plt
import seaborn as sns

# Visualize coefficient heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(clf.coef_, cmap="coolwarm", center=0, xticklabels=False)
plt.xlabel("One-hot features (512 × 4)")
plt.ylabel("EC Label Index")
plt.title("Logistic Regression Weights (One-hot DNA)")
plt.tight_layout()
plt.show()


## K-mer Count Feature Variant (e.g., 4-mers)


In [ ]:
from collections import Counter
from itertools import product

# Vocabulary of all 4-mers
k = 4
kmer_vocab = [''.join(p) for p in product("ACGT", repeat=k)]

def kmer_features_dna(seq, k=4):
    kmers = [seq[i:i+k] for i in range(len(seq)-k+1)]
    counts = Counter(kmers)
    return np.array([counts.get(kmer, 0) for kmer in kmer_vocab])

In [ ]:
X_train_kmer = np.stack(df_train["Sequence"].map(kmer_features_dna))
X_test_kmer = np.stack(df_test["Sequence"].map(kmer_features_dna))

print("Shape of k-mer feature matrix:", X_train_kmer.shape)


In [ ]:
#retrain log reg
clf_kmer = LogisticRegression(
    multi_class='multinomial',
    solver='saga',
    max_iter=1000,
    n_jobs=-1,
    verbose=1
)
clf_kmer.fit(X_train_kmer, y_train)

y_pred_kmer = clf_kmer.predict(X_test_kmer)
print("K-mer Accuracy:", accuracy_score(y_test, y_pred_kmer))
print("K-mer F1-score (macro):", f1_score(y_test, y_pred_kmer, average='macro'))


## cross val evaluation

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(
    clf,
    X_train,
    y_train,
    cv=5,
    scoring='f1_macro',
    n_jobs=-1
)

print("5-fold cross-validated F1 scores:", scores)
print("Mean F1:", scores.mean())


## blast based evaluation

In [ ]:
#step 1: write sequences to fasta
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio import SeqIO

# Write training sequences
train_records = [
    SeqRecord(Seq(seq), id=entry, description=label)
    for seq, entry, label in zip(df_train["Sequence"], df_train["Entry"], df_train["Label"])
]
SeqIO.write(train_records, "train_set.fasta", "fasta")

# Write test sequences
test_records = [
    SeqRecord(Seq(seq), id=entry, description=label)
    for seq, entry, label in zip(df_test["Sequence"], df_test["Entry"], df_test["Label"])
]
SeqIO.write(test_records, "test_set.fasta", "fasta")


In [ ]:
# step 2 : create BLAST DB
makeblastdb -in train_set.fasta -dbtype nucl -out train_db


In [ ]:
# step 3- run blast
blastn -query test_set.fasta -db train_db -outfmt "6 qseqid sseqid pident length" -out blast_results.txt


In [ ]:
#step 4: parse blast files
blast_df = pd.read_csv("blast_results.txt", sep="\t", header=None, names=["query", "subject", "pident", "length"])

# Take best hit per test query
top_hits = blast_df.sort_values(["query", "pident", "length"], ascending=[True, False, False]).drop_duplicates("query")

# Map training subject ID to EC label
entry_to_label = dict(zip(df_train["Entry"], df_train["Label"]))
predicted_labels = top_hits["subject"].map(entry_to_label)
true_labels = df_test.set_index("Entry").loc[top_hits["query"], "Label"].values

# Evaluate
print("BLAST Accuracy:", accuracy_score(true_labels, predicted_labels))
print("BLAST Macro F1:", f1_score(true_labels, predicted_labels, average='macro'))
